In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from operator import itemgetter
from langchain.schema import StrOutputParser

In [2]:
loader = TextLoader('data/sample.txt')
document = loader.load()
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.8)

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
documents = text_splitter.split_documents(document)

In [4]:
db = FAISS.from_documents(documents, embedding=OpenAIEmbeddings())
retriever = db.as_retriever()

In [5]:
template = """You are an assistant who reads news and summarize for executives. Use the context given below to answer the question.
If you don't have enough information to provid an answer, just say don't have enough information.
Context:
{context}

Question:
{question}"""
prompt = PromptTemplate(input_variables=["context", "question"], template=template)
chain = {"context": itemgetter("context"), "question": itemgetter("question")} | prompt | model | StrOutputParser()
response = chain.invoke({"context": itemgetter("question") | retriever, "question": "Give me the summary of the article"})
print(response)


The article discusses the LangChain AI framework and its impact on the AI ecosystem. It mentions that LangChain is becoming extremely popular and is causing harm to the development and code clarity of other AI frameworks. The article highlights criticisms of LangChain, including unnecessary abstraction, poor documentation, lack of customizability, difficulty debugging, and rapid updates. However, some commenters mention benefits such as easy component swapping and potential future improvements. Overall, sentiment towards LangChain for real production systems is negative, with many users reverting to coding LLM workflows directly. Nonetheless, its popularity makes it a safe choice for initial development.
